In [ ]:

import numpy as np
import os

CGMD_dir = '../data/CG'
processed_data_path = '../data/DP'

force = np.load(CGMD_dir+'/force.npy')
position = np.load(CGMD_dir+'/pos.npy')
types = np.load(CGMD_dir+'/atom_types.npy')[0,:]
types = (types-1).astype(int)

num_frames = force.shape[0]
force = force.reshape(num_frames,-1)

# make the unit of force to be eV/A
force = force*0.0433634

position = position.reshape(num_frames,-1)
print('force shape: ', force.shape)
print('position shape: ', position.shape)
print('types shape: ', types.shape)
#generate train_index and val_index

index = np.arange(num_frames)
np.random.shuffle(index)
train_index = index[:int(num_frames*0.9)]
val_index = index[int(num_frames*0.9):]

n_train = len(train_index)
n_val = len(val_index)

model_name = 'graph.pb'

print('n_train: ', n_train)
print('n_val: ', n_val)

#show max and min of force
print('max force: ', np.max(force))
print('min force: ', np.min(force))

#prepare train data
train_path = processed_data_path+'/training_data'
train_set_path = train_path+'/set.000'

#创建文件夹
if not os.path.exists(train_path):
    os.mkdir(train_path)
if not os.path.exists(train_set_path):
    os.mkdir(train_set_path)

#write type.raw, each line is the type of one atom
with open(train_path+'/type.raw', 'w') as f:
    for i in range(len(types)):
        f.write(str(int(types[i]))+' ')

np.save(train_set_path+'/force.npy', force[train_index])
np.save(train_set_path+'/coord.npy', position[train_index])

val_path = processed_data_path+'/validation_data'
val_set_path = val_path+'/set.000'

if not os.path.exists(val_path):
    os.mkdir(val_path)
if not os.path.exists(val_set_path):
    os.mkdir(val_set_path)

with open(val_path+'/type.raw', 'w') as f:
    for i in range(len(types)):
        f.write(str(int(types[i]))+' ')

np.save(val_set_path+'/force.npy', force[val_index])
np.save(val_set_path+'/coord.npy', position[val_index])

box = np.load(CGMD_dir+'/box.npy')
print(box.shape)
box_matrix = np.zeros((num_frames, 3, 3))
for i in range(num_frames):
    np.fill_diagonal(box_matrix[i], box[i])

print("cutoff:", np.min(box)/2)

#save box_matrix as npy
np.save(train_set_path+'/box.npy', box_matrix[train_index])
np.save(val_set_path+'/box.npy',  box_matrix[val_index])

In [ ]:
os.system(f"(cd {processed_data_path} && dp train input.json)")
os.system(f"(cd {processed_data_path} && dp freeze -o ./model/graph.pb)")
